# 自定义基础：张量和操作

这是TensorFlow入门教程，显示如何:

* 导入需要的包
* 创建和使用张量
* 使用GPU加速
* 展示`tf.data.Dataset`

## 导入TensorFlow

首先，导入`tensorflow`模块。从TensorFlow 2开始，默认情况下将打开即刻执行功能。这样可以使TensorFlow更具交互性，我们将在以后详细讨论其细节。

In [ ]:
import tensorflow as tf

## 张量

`tf.Tensor`张量是一个多维数组。与NumPy `ndarray`对象相似， `tf.Tensor`对象具有数据类型和形状。此外， `tf.Tensor`可以驻留在加速器内存（如GPU）中。 TensorFlow提供了丰富的操作库（ tf.add ， tf.matmul ， tf.linalg.inv等），这些操作消耗并产生`tf.Tensor` 。这些操作会自动转换本地Python类型，例如：


In [ ]:
print(tf.add(1, 2))  # 自动转换python类型
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# 支持重载运算符
print(tf.square(2) + tf.square(3))

每个`tf.Tensor`有形状和数据类型:

In [ ]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

Numpy数组和张量的最明显的区别是：

1. 张量可由加速器存储，如GPU和TPU支持.
2. 张量是不可变.

### NumPy兼容性

在TensorFlow的tf.Tensor和NumPy的ndarray之间进行转换很容易：
* TensorFlow操作会自动将NumPy ndarray转换为Tensors。
* NumPy操作自动将张量转换为NumPy ndarray。

使用张量的`.numpy()`方法将张量显式转换为NumPy ndarray。这些转换通常很便宜，因为如果可能的话，数组和tf.Tensor共享底层的内存表示形式。但是，共享底层表示并不总是可能的，因为tf.Tensor可能托管在GPU内存中，而NumPy数组始终由主机内存支持，并且转换涉及从GPU到主机内存的复制。


In [ ]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

## GPU加速

使用GPU进行计算可加速许多TensorFlow操作。没有任何标记，TensorFlow会自动决定是使用GPU还是CPU进行操作-必要时在CPU和GPU内存之间复制张量。由操作产生的张量通常由执行操作的设备的内存支持，例如：


In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

### 设备名

`Tensor.device`属性提供存放张量内容的设备的标准字符串名称。此名称编码许多详细信息，例如正在执行此程序的主机的网络地址以及该主机中的设备的标识符。这是分布式执行TensorFlow程序所必需的。如果张量放置在主机的第N个GPU上，则字符串以`GPU:<N>`结尾。


### 显式设备放置

在TensorFlow中， `放置`是指如何将单个操作分配（放置）在设备上以执行。如前所述，当没有提供明确的指导时，TensorFlow会自动决定执行哪个设备，并在需要时将张量复制到该设备上。但是，可以使用`tf.device`上下文管理器将TensorFlow操作显式放置在特定设备上，例如：


In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# 强制在CPU上执行
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# 如果可能，强制在1号CPU上执行
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## 数据集

这一节使用[`tf.data.Dataset` API] 构建喂数据到模型的管道。`tf.data.Dataset` API用来从简单的可复用代码构建给训练和评价循环使用的高效的、复杂的输入管道。

### 创建`Dataset`

使用工厂方法（例如Dataset.from_tensors ， Dataset.from_tensor_slices ）或使用从文件读取的对象（例如TextLineDataset或TFRecordDataset)来创建`源数据集`。有关更多信息，请参见TensorFlow数据集指南 。


In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# 创建一个CSV文件
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### 应用转换

使用操作，如[`map`], [`batch`], 和 [`shuffle`]对数据集中的记录进行转换。 

In [ ]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### 迭代

`tf.data.Dataset`对象支持在记录集上迭代:

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)